In [1]:
from taxcalc import *
from bokeh.io import show, output_notebook
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

## Setting up the calculator

In [2]:
# Create Records and Policy objects for calculator
recs = Records()
pol = Policy()
# Create Calculator object
calc = Calculator(records=recs, policy=pol)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [3]:
# Advance calculator to 2017 and run
calc.advance_to_year(2017)
calc.calc_all()

## Displaying baseline results

In [4]:
# Diagnostic Table
utils.create_diagnostic_table(calc)

,2017
Returns (#m),170.4
AGI ($b),"11,440.8"
Itemizers (#m),44.8
Itemized Deduction ($b),"1,268.7"
Standard Deduction Filers (#m),111.9
Standard Deduction ($b),996.2
Personal Exemption ($b),"1,175.7"
Taxable Income ($b),"8,492.6"
Regular Tax ($b),"1,777.1"
AMT Income ($b),"10,822.0"


In [5]:
# Distribution Table
utils.create_distribution_table(calc.records, 'weighted_deciles', 'weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,694","17,041,816","-6,948","11,516,212","6,294","4,682",21,"4,443",58,7,"-7,360",0,0,7,0,7,137,-124,340,216
1,"8,807","17,040,415","6,002","14,010,959","7,370","53,927",27,"4,948",315,28,"5,974","14,445",0,28,1,5,743,-710,844,134
2,"15,943","17,041,505","10,265","13,678,936","8,082","312,986",210,"6,559","1,631",160,"10,122","16,510",1,161,20,7,"1,634","-1,486","1,469",-17
3,"23,438","17,044,022","15,739","14,832,468","8,336","731,409",590,"6,978","4,341",463,"15,373","35,460",2,465,93,13,"1,414","-1,028","2,105","1,077"
4,"32,816","17,041,666","24,306","14,799,151","8,250","1,706,190","1,483","7,292","10,012","1,162","23,372","19,038",2,"1,164",255,18,974,-47,"3,080","3,034"
5,"44,317","17,042,127","35,697","13,772,526","7,913","3,146,508","2,982","7,429","18,933","2,333","33,909","25,706",4,"2,337",405,30,419,"1,544","4,233","5,776"
6,"59,254","17,040,903","50,971","11,779,967","7,175","5,241,174","5,537","7,768","30,964","4,164","47,677","25,635",5,"4,169",511,45,122,"3,580","5,685","9,266"
7,"81,104","17,042,185","74,150","9,416,430","6,147","7,617,314","8,813","8,592","50,717","7,482","69,308","72,693",8,"7,490",655,68,77,"6,826","8,400","15,226"
8,"120,974","17,042,904","112,646","5,994,841","4,253","11,044,938","15,074","10,184","83,182","13,455","104,600","253,699",32,"13,487",796,105,95,"12,700","13,770","26,470"
9,"367,045","17,041,967","348,501","2,092,487","1,497","14,937,255","39,706","9,557","298,179","75,025","332,039","5,108,152","2,582","77,607","1,643","2,566",47,"78,483","24,237","102,721"


## Implementing a policy reform

### Steps for modeling a reform
1. Create a new calculator object
2. Specify reform provisions
3. Apply the reform to the policy object
4. Run the calculator with the specified reform

#### Step 1: Create new calculator object

In [6]:
rec_x = Records()
pol_x = Policy()
calc_x = Calculator(records=rec_x, policy=pol_x)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


#### Step 2: Specify reform provisions
Reforms are formated as dictionaries with the following structure:
```python
reform = {
    year: {
        'parameter 1': [new value],
        'parameter 2': [new value],
        .
        .
        .
        'parameter x': [new value]
     }
}
```
The parameter names can be found in the file `current_law_policy.json`

In [7]:
# Sample reform doubling the standard deduction, eliminating the NIIT, Implement the Fair Share Tax 
reform = {
    2017: {
        '_STD': [[12700, 25400, 12700, 18700, 25400]],
        '_NIIT_rt': [0.0],
        '_FST_AGI_trt': [0.3]
    }
}

#### Step 3: Apply reform to policy object

The `implement_reform()` method within the policy class reads the dictionary specifying the reform and changes the values of the parameters read from `current_law_policy.json`

In [8]:
pol_x.implement_reform(reform)

#### Step 4: Run the calculator with the reforms

In [9]:
calc_x.advance_to_year(2017)
calc_x.calc_all()

#### View results

In [10]:
utils.create_diagnostic_table(calc_x)

,2017
Returns (#m),170.4
AGI ($b),"11,440.8"
Itemizers (#m),23.3
Itemized Deduction ($b),926.1
Standard Deduction Filers (#m),133.4
Standard Deduction ($b),"2,373.1"
Personal Exemption ($b),"1,175.7"
Taxable Income ($b),"7,847.6"
Regular Tax ($b),"1,673.5"
AMT Income ($b),"10,999.4"


In [11]:
utils.create_distribution_table(calc_x.records, 'weighted_deciles', 'weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,694","17,041,816","-6,948","11,517,984","11,699","2,911",17,"4,443",54,7,"-7,359",0,0,7,0,6,137,-123,340,217
1,"8,807","17,040,415","6,002","14,039,581","13,648","25,305",9,"4,948",69,3,"5,985","14,445",0,4,0,5,743,-734,844,110
2,"15,943","17,041,505","10,265","13,951,861","15,938","40,061",26,"6,559",149,13,"10,237","18,791",1,14,1,7,"1,635","-1,615","1,469",-146
3,"23,438","17,044,022","15,739","15,352,097","16,623","211,779",218,"6,978","1,698",167,"15,599","35,460",3,169,24,13,"1,449","-1,290","2,105",815
4,"32,816","17,041,666","24,306","16,079,445","17,182","425,895",518,"7,292","5,416",601,"23,960","19,038",2,604,100,18,"1,084",-562,"3,080","2,518"
5,"44,317","17,042,127","35,697","15,872,934","17,400","1,046,100","1,396","7,429","13,034","1,571","34,832","26,174",4,"1,575",246,30,543,817,"4,233","5,049"
6,"59,254","17,040,903","50,971","15,163,641","17,643","1,857,500","2,731","7,768","24,081","3,101","49,289","28,317",7,"3,107",415,45,199,"2,538","5,685","8,223"
7,"81,104","17,042,185","74,150","13,892,391","17,527","3,141,985","4,742","8,592","43,442","6,254","71,479","191,320",12,"6,265",634,68,92,"5,608","8,400","14,008"
8,"120,974","17,042,904","112,646","12,065,764","16,832","4,974,014","8,797","10,184","76,880","12,128","107,716","471,749",41,"12,169",794,105,95,"11,385","13,770","25,155"
9,"367,045","17,041,967","348,501","5,454,125","7,841","11,575,382","35,887","9,557","295,661","74,351","333,684","5,863,564","2,677","77,029","1,642",187,47,"79,314","24,237","103,551"


## Comparing reform results to the baseline
There are a few ways to compare reform results. Some of the most popular are difference tables and MTR/ATR Plots

#### Differences Table

In [12]:
utils.create_difference_table(calc.records, calc_x.records, 'weighted_deciles', income_to_present='combined')

,tax_cut,tax_inc,count,mean,tot_change,perc_inc,perc_cut,share_of_change
0,"3,048",92,"17,041,816",1,"12,844,020",0.00%,0.02%,-0.02%
1,"2,603,855",0,"17,040,415",-24,"-407,013,712",0.00%,15.28%,0.54%
2,"5,158,766",0,"17,041,505",-129,"-2,199,696,451",0.00%,30.27%,2.93%
3,"7,928,076",0,"17,044,022",-262,"-4,464,320,090",0.00%,46.52%,5.95%
4,"11,879,798",0,"17,041,666",-516,"-8,789,612,572",0.00%,69.71%,11.71%
5,"13,243,230",0,"17,042,127",-727,"-12,391,824,216",0.00%,77.71%,16.51%
6,"14,222,319",0,"17,040,903","-1,043","-17,767,934,877",0.00%,83.46%,23.68%
7,"13,760,550",0,"17,042,185","-1,218","-20,763,475,905",0.00%,80.74%,27.67%
8,"12,057,048",11,"17,042,904","-1,316","-22,421,224,622",0.00%,70.75%,29.88%
9,"10,001,031","161,544","17,041,967",831,"14,157,296,698",0.95%,58.68%,-18.87%


#### Marginal and Average Tax Rate Plots

In [13]:
# Create Marginal Tax Rate and Average Tax Rate Plots
mtr_data = utils.mtr_graph_data(calc, calc_x)
atr_data = utils.atr_graph_data(calc, calc_x)

In [14]:
mtr_plot = utils.xtr_graph_plot(mtr_data)
atr_plot = utils.xtr_graph_plot(atr_data)
show(column([mtr_plot, atr_plot]))

## Behavioral Reform
Tax-Calculator also has the capability to perform partial equlibrium simulation dynamic analysis.

Steps for behavioral analysis:
1. Create new calculator object
2. Specify and implement policy reforms for new calculator
3. Specify and implement assumed behavioral elasticies
4. Use `response` method in the bahavior class to get a new calculator with the elasticies applied

Behavioral reforms are specified in the same structure as policy reforms:

```python
behavior_reform = {
    year: {
        '_BE_inc': [new value],
        '_BE_sub': [new value]
    }
}
```
Elasticity parameters are found in `behavior.json`

#### Step 1: Create new calculator object

In [15]:
# This is the same as every other calculator, but this time a behavior object is also created
rec_b = Records()
pol_b = Policy()
beh_b = Behavior()
calc_b = Calculator(records=rec_b, policy=pol_b, behavior=beh_b)
calc_b.advance_to_year(2017)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


#### Step 2: Specify  and implement policy reforms

In [16]:
# We'll use the same policy reform as before
pol_b.implement_reform(reform)

#### Step 3: Specify and implement assumed elasticities
The `update_behavior` method in the `Behavior` class updates the parameters read from `behavior.json` with user specified values

In [17]:
# Set income elasticy to -0.2 and substiution elasticity to 0.4
behavior_reform = {
    2017: {
        '_BE_inc': [-0.2],
        '_BE_sub': [0.4]
    }
}
beh_b.update_behavior(behavior_reform)

#### Step 4: Use `response` method in Behavior class to get new calculator

In [18]:
calc_response = Behavior.response(calc, calc_b)

#### Compare policy reform with a behavior response to the baseline

In [19]:
utils.create_difference_table(calc.records, calc_response.records, 'weighted_deciles')

,tax_cut,tax_inc,count,mean,tot_change,perc_inc,perc_cut,share_of_change
0,"3,053",88,"17,041,816",1,"18,819,721",0.00%,0.02%,-0.04%
1,"2,603,855",0,"17,040,415",-24,"-406,097,032",0.00%,15.28%,0.76%
2,"5,158,766",0,"17,041,505",-129,"-2,200,420,045",0.00%,30.27%,4.14%
3,"7,922,919","5,157","17,044,022",-257,"-4,387,977,638",0.03%,46.49%,8.25%
4,"11,820,108","59,690","17,041,666",-514,"-8,760,557,362",0.35%,69.36%,16.48%
5,"13,176,539","66,691","17,042,127",-721,"-12,294,551,348",0.39%,77.32%,23.12%
6,"13,980,971","241,348","17,040,903",-986,"-16,799,405,410",1.42%,82.04%,31.60%
7,"13,667,511","93,039","17,042,185","-1,236","-21,066,706,363",0.55%,80.20%,39.62%
8,"11,861,572","195,487","17,042,904","-1,240","-21,140,931,357",1.15%,69.60%,39.76%
9,"9,512,477","650,098","17,041,967","1,987","33,867,381,940",3.81%,55.82%,-63.70%


In [20]:
mtr_data_b = utils.mtr_graph_data(calc, calc_response)
atr_data_b = utils.atr_graph_data(calc, calc_response)

In [21]:
mtr_plot_b = utils.xtr_graph_plot(mtr_data_b)
atr_plot_b = utils.xtr_graph_plot(atr_data_b)
show(column([mtr_plot_b, atr_plot_b]))

In [22]:
utils.create_diagnostic_table(calc_response)

,2017
Returns (#m),170.4
AGI ($b),"11,527.3"
Itemizers (#m),23.3
Itemized Deduction ($b),928.5
Standard Deduction Filers (#m),133.4
Standard Deduction ($b),"2,373.2"
Personal Exemption ($b),"1,175.6"
Taxable Income ($b),"7,929.1"
Regular Tax ($b),"1,701.4"
AMT Income ($b),"11,082.5"


In [28]:
calc.records.payrolltax

array([ 25773.36588365,   1986.25549625,  27219.130578  , ...,
            0.        ,      0.        ,      0.        ])